In [1]:
import pickle
import json
import argparse
from tqdm import tqdm
from copy import deepcopy
from queue import PriorityQueue
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'


In [2]:
import pandas as pd
ans = pd.read_json('ans.json')
my_ans = pd.read_json('ans1.json')

In [3]:
with open('data/data.pkl', 'rb') as fp:
        data = pickle.load(fp)
with open('data/phoneme_table.json', 'r') as fp:
        phenome_table = json.load(fp)
with open('data/vocabulary.json', 'r') as fp:
        vocabulary = json.load(fp)
class CostModel(object):
    def __init__(self) -> None:
        # Load Whisper model and processor
        self.__processor = WhisperProcessor.from_pretrained("openai/whisper-small.en")
        self.__model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small.en").to(DEVICE)
        self.__audio_inputs = None

    def set_audio(self, audio, sampling_rate):
        self.__audio_inputs = self.__processor(
            audio, sampling_rate=sampling_rate, return_tensors="pt"
        ).input_features.to(DEVICE)

    def get_loss(self, text):
        # Prepare the target text input IDs
        target = self.__processor(
            text=text, return_tensors="pt", padding=True
        ).input_ids.to(DEVICE)

        with torch.no_grad():
            outputs = self.__model(input_features=self.__audio_inputs, labels=target)

        return outputs.loss.item()


class Environment(object):
    def __init__(self, init_state, cost_function, phoneme_table) -> None:
        self.init_state = init_state
        self.phoneme_table = deepcopy(phoneme_table)
        self.__cost_function = cost_function

    def compute_cost(self, text):
        # try:
        cost = self.__cost_function(text)
        # except:
        #     cost = 1e6
        return cost
replacement_lens = []
matrix = {}
for char in phenome_table:
	for replacement in phenome_table[char]:
		if(replacement not in matrix):
			matrix[replacement] = []
		matrix[replacement].append(char)

for rep in matrix:
    if(len(rep) not in replacement_lens):
        replacement_lens.append(len(rep))



In [ ]:
l = [6,]

In [23]:
final = []
audio = None
sr = None
text = None
pred = None
corrected_texts = []
cost_model = CostModel()
for sample in tqdm(data[:67]):
        audio = sample['audio']['array']
        sr = sample['audio']['sampling_rate']
        text = sample['text']
        cost_model.set_audio(audio, sr)
        # print(text)
        # environment = Environment(text, cost_model.get_loss, phenome_table)

        # # try:
        # agent.asr_corrector(environment)
        # pred = agent.best_state
        # except:
        #     pred = None
        corrected_texts.append(text)
        # break
cost_model.set_audio(audio,sr)
environment = Environment(text, cost_model.get_loss, phenome_table)

def cost(text):
    return environment.compute_cost(text)

start_state = text
words = start_state.split()
beam = []
wbeams =[[word] for word in words ]
wpq =[PriorityQueue() for _ in words]
d = [{} for _ in words]
best_words = [None for _ in words]
def optimize_word(word,idx,epsilon,beam_size=30000,beam_depth=2,best_n=20):
    best_word  = word
    global current_state
    best_cost = cost(current_state)
    initial_word = word
    inital_cost = best_cost
    wpq[idx].put((best_cost,best_word))
    d[idx][best_word] = best_cost
    f = 1 +epsilon*len(best_word)
    count = 0
    for depth in range(beam_depth):
        stop = False
        queue = PriorityQueue()
        new_beam = []
        for current_word in wbeams[idx]:
            for l in replacement_lens:
                for j in range(len(current_word)):
                    if(j+l>len(current_word)): continue
                    to_replace = current_word[j:j+l]
                    if to_replace in matrix:
                        for replacement in matrix[to_replace]:
                            new_word = current_word[:j]+replacement + current_word[j+l:]
                            words[idx] = new_word
                            new_sentence = ' '.join(words)
                            if(new_word not in d[idx]):
                                c = cost(new_sentence)
                                d[idx][new_word] = c
                                queue.put((c,new_word))
                                wpq[idx].put((c,new_word))
                                if(c<best_cost):
                                    best_cost = c
                                    best_word = new_word
                                    count = 0
                                else:
                                    count +=1
                                if(count>=100):
                                    stop = True
                                # print((c,new_sentence,best_word,len(wbeams[idx])))
        if(stop):
            break
        next_beam_size = 0
        while((next_beam_size<beam_size) and not queue.empty()):
            word_cost,beam_word, = queue.get()
            if(word_cost<f*best_cost):
                new_beam.append(beam_word)
                next_beam_size+=1
                if(word_cost<best_cost):
                    best_cost = word_cost
                    best_word = beam_word
            else: break
        wbeams[idx] = new_beam
        print((count,initial_word,inital_cost,best_word,best_cost))
        if(count>=100 and initial_word == best_word):
            break

    ans = []
    a = wpq[idx].get()
    ans.append(a[1])
    u = []
    for _ in range(best_n):
        if not wpq[idx].empty():
            cw  = wpq[idx].get()
            if(a[0]*f> cw[0]):
                ans.append(cw[1])
            else: break
        else: break
    best_words[idx] = ans
    words[idx] = ans[0]
    current_state = ' '.join(words)


current_state = start_state 
for i,w in enumerate(start_state.split()):
    optimize_word(w,i,beam_size=300,beam_depth=3,epsilon=0.020,best_n=10)
    words[i] = best_words[i][0]
best_cost = cost(current_state)

prq = PriorityQueue()
beam_depth = 4
beam_size = 20

for _ in range(beam_depth):
    bs_words = current_state.split(' ')
    best_cost = cost(current_state)
    for i, word in enumerate(bs_words):
        replacement_words = best_words[i]
        current_word = word
        for word1 in replacement_words:
            bs_words[i] = word1
            sentence_potential = ' '.join(bs_words)
            cost_potential = cost(sentence_potential)
            prq.put((-1*cost_potential,sentence_potential))
            if(prq.qsize()>beam_size):
                prq.get()
            if cost_potential<best_cost:
                best_cost = cost_potential
                current_state = sentence_potential
                current_word = bs_words[i]
        bs_words[i] = current_word

k = 10
current_sentence = current_state
best_state = current_state
prq1 = PriorityQueue()
best_cost = cost(best_state)
for word in vocabulary:
	new_sentence = word + " " + current_sentence
	new_cost = cost(new_sentence)
	if new_cost < best_cost:
		best_state = new_sentence
		best_cost = new_cost
	prq1.put((-1*new_cost,word))
	if prq1.qsize() >= k:
		prq1.get()

prq2 = PriorityQueue()
for word in vocabulary:
	new_sentence = current_sentence  + " " + word
	new_cost = cost(new_sentence)
	if new_cost < best_cost:
		best_state = new_sentence
		best_cost = new_cost
	prq2.put((-1*new_cost,word))
	if prq2.qsize() >= k:
		prq2.get()

list1 = [prq1.get()[1] for _ in range(prq1.qsize())]
list2 = [prq2.get()[1] for _ in range(prq2.qsize())]

for elem1 in list1:
	for elem2 in list2:
		new_sentence = elem1 + " "+current_sentence + " "+elem2
		print(new_sentence)
		new_cost = cost(new_sentence)
		if new_cost < best_cost:
			best_state = new_sentence
			best_cost = new_cost
final.append(best_state)
# current_state
# for 6,w in enumerate(words):
#     for x in best_words[i]:
#         words[i] = x
#         new_sentence=  ' '.join(words)
#         c = cost(new_sentence)
#         if(c < best_cost):
#             best_cost = c
#             current_state = new_sentence
#         else:
#             words[i]  = w

100%|██████████| 67/67 [00:00<00:00, 182.33it/s]


(4, 'HER', 3.668325424194336, 'HER', 3.668325424194336)
(4, 'HER', 3.668325424194336, 'HER', 3.668325424194336)
(4, 'HER', 3.668325424194336, 'HER', 3.668325424194336)
(4, 'FADER', 3.668325424194336, 'FATHER', 3.424816846847534)
(14, 'FADER', 3.668325424194336, 'FATHER', 3.424816846847534)
(63, 'FADER', 3.668325424194336, 'FATHER', 3.424816846847534)
(2, 'NOW', 3.424816846847534, 'NAUW', 3.1305131912231445)
(4, 'NOW', 3.424816846847534, 'NAUR', 3.117487668991089)
(12, 'NOW', 3.424816846847534, 'NAUR', 3.117487668991089)
(11, 'NISTER', 3.117487668991089, 'MISTER', 2.6546361446380615)
(20, 'NISTER', 3.117487668991089, 'MISTER', 2.6546361446380615)
(51, 'NISTER', 3.117487668991089, 'MISTER', 2.6546361446380615)
(13, 'ALLEN', 2.6546361446380615, 'ALLEN', 2.6546361446380615)
(70, 'ALLEN', 2.6546361446380615, 'ALLEN', 2.6546361446380615)
(70, 'ALLEN', 2.6546361446380615, 'ALLEN', 2.6546361446380615)
(5, 'DID', 2.6546361446380615, 'DID', 2.6546361446380615)
(14, 'DID', 2.6546361446380615, 'DI

In [26]:
best_words

[['HER'],
 ['FATHER',
  'FAITHER',
  'FADER',
  'FAUTHER',
  'FATHHER',
  'FATHIER',
  'FAETHER',
  'FATHHAER',
  'FATHHAIR'],
 ['NAUR', 'NAUW', 'NAUUR', 'NOR'],
 ['MISTER', 'MISTEUR', 'MISTAIR', 'MISTAER'],
 ['ALLEN', 'ALLIEN', 'ALLEM', 'ARLEN', 'ALLAEN', 'ALLAIN'],
 ['DID', 'DIED'],
 ['SO'],
 ['IT'],
 ['WAS'],
 ['CERTAINLY', 'CEURTAINLY'],
 ['VERY', 'VEURY', 'VAIRY']]

In [24]:
final[0]

'NEITHER HER FATHER NOR MISTER ALLEN DID SO IT WAS CERTAINLY VERY PROVOKING'

In [21]:
avg

118.79040277004242

In [18]:
a

117.1194543838501

In [20]:
l = [11, 15, 17, 20, 26, 29]
data = [data[i] for i in l]

In [21]:
data_subset

[{'audio': {'array': array([-1.61743164e-03, -1.37329102e-03, -9.76562500e-04, ...,
          -9.15527344e-05, -3.05175781e-05, -1.52587891e-04]),
   'sampling_rate': 16000},
  'gold': 'ALL READY THEY ANSWERED AND THE LIEUTENANT LED THE WAY TO THE TRAIN',
  'text': 'ALL READY THEY ANSWERED AND THE LIOTENANT LED FE WAY TO THE TLAIN'},
 {'audio': {'array': array([-0.00152588,  0.00259399,  0.00375366, ..., -0.00094604,
          -0.00021362,  0.00079346]),
   'sampling_rate': 16000},
  'gold': 'DAUPHINES WITHOUT A RIGHT OR WRONG SIDE IN THE PIECE',
  'text': 'DAUPHINES WITHOUT A RIGHT OR WRONG SIDE IN TE PIECE'},
 {'audio': {'array': array([-0.02770996, -0.02893066, -0.02896118, ..., -0.0050354 ,
          -0.00552368, -0.00509644]),
   'sampling_rate': 16000},
  'gold': 'AND THEN COMES MURDER CREEK WHICH TAKES YOU ON TO TARRANGOWER',
  'text': 'AND FEN COMEZ MURDEW CLEEK WHICH TAKES YOU ON TO TARRANGOWER'},
 {'audio': {'array': array([0.        , 0.        , 0.        , ..., 0.00088501,

In [80]:
k = 10
current_sentence = current_state
best_state = current_state
prq1 = PriorityQueue()
best_cost = cost(best_state)
for word in vocabulary:
	new_sentence = word + " " + current_sentence
	new_cost = cost(new_sentence)
	if new_cost < best_cost:
		best_state = new_sentence
		best_cost = new_cost
	prq1.put((-1*new_cost,word))
	if prq1.qsize() >= k:
		prq1.get()

prq2 = PriorityQueue()
for word in vocabulary:
	new_sentence = current_sentence  + " " + word
	new_cost = cost(new_sentence)
	if new_cost < best_cost:
		best_state = new_sentence
		best_cost = new_cost
	prq2.put((-1*new_cost,word))
	if prq2.qsize() >= k:
		prq2.get()

list1 = [prq1.get()[1] for _ in range(prq1.qsize())]
list2 = [prq2.get()[1] for _ in range(prq2.qsize())]

for elem1 in list1:
	for elem2 in list2:
		new_sentence = elem1 + " "+current_sentence + " "+elem2
		print(new_sentence)
		new_cost = cost(new_sentence)
		if new_cost < best_cost:
			best_state = new_sentence
			best_cost = new_cost


HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CARNIVAL
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CONSCIOUSNESS
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF FATHER
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF KNOWLEDGE
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CHANNEL
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CHAPTER
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF SUCCESS
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CHAMBERS
HESITATION WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF BABYLON
COUNTRY WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CARNIVAL
COUNTRY WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF CONSCIOUSNESS
COUNTRY WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF FATHER
COUNTRY WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF KNOWLEDGE
COUNTRY WITH A HEART OVERWHELM'D WITH DESPAIR 

In [81]:
best_state

"RETURN'D WITH A HEART OVERWHELM'D WITH DESPAIR TO THE COURT OF BABYLON"

CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS EXPOSING
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS CONSISTENT
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS COUNTRY
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS SOCIETY
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS CHANNEL
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS KNOWLEDGE
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS CHAMBERS
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS CHARACTER
CHARACTER THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS RESULT
CARNIVAL THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS EXPOSING
CARNIVAL THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS CONSISTENT
CARNIVAL THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS COUNTRY
CARNIVAL THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS SOCIETY
CARNIVAL THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL 

In [74]:
best_state

'EXPOSING THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS RESULT'

In [62]:
current_state

'THE CROWN ALTHOUGH THEIR CONCORD HAD ONLY EVIL AS ITS'

'DAUPHINES WITHOUT A RIGHT OR WRONG SIDE IN THE PIECE'

In [46]:
current_state

'ALL READY THEY ANSWERED AND THE LIEUTENANT LED THE WAY TO THE TRAIN'

In [2]:
beam_depth= 1
beam_size = 20
best_state = current_state
beam = set()
beam.add(current_state)
vis = set()
vis.add(current_state)
words = current_state.split()
for _ in range(beam_depth):
	prq = PriorityQueue()
	for cs in beam:
		cs_words = cs.split(' ')
		for i,word in enumerate(cs_words):
			for pos_rep in best_words[i]:
				new_sol = ' '.join(cs_words[:i]+[pos_rep]+cs_words[i+1:])
				if(new_sol in vis): continue
				else: vis.add(new_sol)
				c = cost(new_sol)
				prq.put((c, new_sol))
		next_beam = set()
		for _ in range(beam_size):
				possol = prq.get()
				if(possol not in next_beam): next_beam.add(possol[1])
				if (possol[0] < cost(best_state)):
					best_cost = possol[0]
					best_state = possol[1]
					current_state = best_state
		beam = next_beam

NameError: name 'PriorityQueue' is not defined

In [9]:
beam

{"AE CAN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "AI CAN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "I CAN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "IAI CAEN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "IAI CAIN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "IAI CAN'T SAY WHETHER THERE IS A WILL OR KNOT LET US TALK OF SOMETHING ELSE",
 "IAI KAAEN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "IAI KAEN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "IAI KAN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE",
 "IE CAN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE"}

"I CAN'T SAY WHETHER THERE IS A WILL OR NOT LET US TALK OF SOMETHING ELSE"